In [ ]:
# Célula 1: Importações e Carregamento de Dados

import pandas as pd
import numpy as np
from datetime import datetime

# Configuração para melhor visualização (opcional)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 20)

# -----------------------------------------------------------
# ATENÇÃO: Carregamento dos Datasets
# Por favor, substitua os nomes de arquivo se necessário e
# certifique-se de que estão no diretório correto.
# -----------------------------------------------------------
try:
    # Tente carregar seus dados reais
    df_clientes = pd.read_csv('clientes.csv')
    df_produtos = pd.read_csv('produtos.csv')
    df_vendas = pd.read_csv('vendas.csv')

    print("✅ Dados de Clientes, Produtos e Vendas carregados com sucesso!")

except FileNotFoundError as e:
    print(f"❌ ERRO: Arquivo não encontrado. Usando DataFrames de Exemplo para demonstração da lógica. Detalhe: {e}")
    # Criação de DataFrames de exemplo (para garantir que o resto do código funcione)
    df_clientes = pd.DataFrame({
        'id_cliente': [1, 2, np.nan, 4, 1, 6], 
        'nome': ['Ana', 'Bruno', 'Carlos', 'Diana', 'Ana', 'Eva'], 
        'email': ['a@a.com', 'b@b.com', 'c@c.com', 'd@d.com', 'a@a.com', 'e@e.com'], 
        'estado': ['SP', 'RJ', 'SP', 'XX', 'RJ', 'MG']
    })
    df_produtos = pd.DataFrame({
        'id_produto': [101, 102, 103, 104, 105], 
        'nome_produto': ['Laptop', np.nan, 'Mouse', 'Teclado', 'Cadeira'], 
        'preco': [1000.50, -50.00, 20.00, 100.00, 300.00], 
        'categoria': ['Eletrônico', 'Livro', 'Eletrônico', 'Roupa', 'Móvel']
    })
    df_vendas = pd.DataFrame({
        'id_venda': [1001, 1002, 1003, 1004, 1005, 1006, 1007], 
        'id_cliente': [1, 2, 999, 4, 1, 6, 777], 
        'id_produto': [101, 102, 103, 105, 101, 999, 104], 
        'quantidade': [1, 0, 3, 2, 1, 5, 1], 
        'valor_unitario': [1000.5, 50.0, 20.0, 150.0, 1000.5, 10.0, 100.0], 
        'valor_total': [1000.5, 0.0, 60.0, 300.0, 1000.5, 50.0, 100.0], # 300.0 deveria ser 2*150=300
        'data_venda': ['2023-01-10', '2023-10-01', '2023-12-31', '2025-01-01', '2023-01-10', '2023-05-20', '2023-01-10'], 
        'status': ['Concluída', 'Concluída', 'Pendente', 'Cancelada', 'Concluída', 'Concluída', 'Concluída']
    })
    print("\n⚠️ A análise a seguir reflete os problemas dos dados de exemplo.")

# Atribuindo nome interno para referências na função
df_clientes._internal_name = 'Clientes'
df_produtos._internal_name = 'Produtos'
df_vendas._internal_name = 'Vendas'

print("\n--- Informações de Clientes (Amostra) ---")
display(df_clientes.head())

In [ ]:
# Célula 2: Identificação e Classificação dos Problemas

problemas_encontrados = []

def registrar_problema(dataset, coluna, dimensao, descricao, afetados):
    """
    Registra um problema de qualidade de dados, calcula o impacto
    e atribui uma criticidade baseada na dimensão e no impacto.
    """
    total_registros = len(dataset)
    impacto_percentual = (afetados / total_registros) * 100
    
    # Heurística de Criticidade:
    # ALTA: Problemas que quebram chaves primárias ou integridade referencial.
    # MÉDIA: Problemas de alto impacto em campos críticos (Completude ou Validade > 5%).
    # BAIXA: Problemas de baixo impacto ou formatacao.
    if dimensao in ['Unicidade', 'Integridade'] and afetados > 0:
        criticidade = 'ALTA'
    elif impacto_percentual >= 5:
        criticidade = 'MÉDIA'
    else:
        criticidade = 'BAIXA'

    problemas_encontrados.append({
        'Dataset': dataset._internal_name,
        'Coluna/Entidade': coluna,
        'Dimensão': dimensao,
        'Descrição': descricao,
        'Registros Afetados': afetados,
        'Impacto (%)': round(impacto_percentual, 2),
        'Criticidade': criticidade
    })

# --- ANÁLISE DE CLIENTES ---
total_clientes = len(df_clientes)
clientes_validos_id = df_clientes['id_cliente'].dropna().astype(int).unique()

# 1. Completude: ID nulo
nulos_id_cliente = df_clientes['id_cliente'].isnull().sum()
if nulos_id_cliente > 0:
    registrar_problema(df_clientes, 'id_cliente', 'Completude', 'IDs de Cliente nulos (Chave Primária)', nulos_id_cliente)

# 2. Unicidade: IDs duplicados
duplicados_total = df_clientes['id_cliente'].duplicated(keep=False).sum()
if duplicados_total > 0:
    registrar_problema(df_clientes, 'id_cliente', 'Unicidade', 'IDs de Cliente duplicados', duplicados_total)
    
# 3. Validade: Estado Inválido (formato de 2 caracteres)
invalidos_estado = df_clientes[df_clientes['estado'].str.len() != 2].shape[0]
if invalidos_estado > 0:
    registrar_problema(df_clientes, 'estado', 'Validade', 'Campo ESTADO com formato diferente de 2 caracteres', invalidos_estado)


# --- ANÁLISE DE PRODUTOS ---
total_produtos = len(df_produtos)
produtos_validos_id = df_produtos['id_produto'].dropna().astype(int).unique()

# 4. Completude: Nome de Produto Nulo
nulos_nome_produto = df_produtos['nome_produto'].isnull().sum()
if nulos_nome_produto > 0:
    registrar_problema(df_produtos, 'nome_produto', 'Completude', 'Nome do Produto nulo', nulos_nome_produto)

# 5. Validade: Preço Negativo (assume-se que preco > 0)
invalidos_preco = df_produtos[df_produtos['preco'] <= 0].shape[0]
if invalidos_preco > 0:
    registrar_problema(df_produtos, 'preco', 'Validade', 'Preço de Produto menor ou igual a zero', invalidos_preco)

# 6. Consistência: Categoria fora do padrão (exemplo de lista válida)
categorias_validas = ['Eletrônico', 'Livro', 'Vestuário', 'Móvel', 'Acessório']
invalidos_categoria = df_produtos[~df_produtos['categoria'].isin(categorias_validas)].shape[0]
if invalidos_categoria > 0:
    registrar_problema(df_produtos, 'categoria', 'Consistência', 'Categoria de Produto fora da lista de valores permitidos', invalidos_categoria)


# --- ANÁLISE DE VENDAS ---
total_vendas = len(df_vendas)

# 7. Integridade: IDs de Cliente órfãos
vendas_com_cliente_invalido = df_vendas[~df_vendas['id_cliente'].isin(clientes_validos_id)].shape[0]
if vendas_com_cliente_invalido > 0:
    registrar_problema(df_vendas, 'id_cliente', 'Integridade', 'Chave estrangeira ID_CLIENTE não existe na tabela Clientes', vendas_com_cliente_invalido)

# 8. Integridade: IDs de Produto órfãos
vendas_com_produto_invalido = df_vendas[~df_vendas['id_produto'].isin(produtos_validos_id)].shape[0]
if vendas_com_produto_invalido > 0:
    registrar_problema(df_vendas, 'id_produto', 'Integridade', 'Chave estrangeira ID_PRODUTO não existe na tabela Produtos', vendas_com_produto_invalido)

# 9. Consistência: valor_total (Regra de Negócio)
df_vendas['valor_calculado'] = df_vendas['quantidade'] * df_vendas['valor_unitario']
# Verifica diferença, considerando pequena margem de erro por float
inconsistentes_valor = df_vendas[np.abs(df_vendas['valor_total'] - df_vendas['valor_calculado']) > 0.01].shape[0]
if inconsistentes_valor > 0:
    registrar_problema(df_vendas, 'valor_total', 'Consistência', 'Valor Total não corresponde a Quantidade × Valor Unitário', inconsistentes_valor)

# 10. Validade: Data Futura
hoje = pd.to_datetime(datetime.now().date())
df_vendas['data_venda'] = pd.to_datetime(df_vendas['data_venda'], errors='coerce')
futuras_venda = df_vendas[df_vendas['data_venda'] > hoje].shape[0]
if futuras_venda > 0:
    registrar_problema(df_vendas, 'data_venda', 'Validade', 'Data de Venda no futuro', futuras_venda)

# 11. Validade: Quantidade Zero ou Negativa (assume-se quantidade >= 1)
invalidos_quantidade = df_vendas[df_vendas['quantidade'] <= 0].shape[0]
if invalidos_quantidade > 0:
    registrar_problema(df_vendas, 'quantidade', 'Validade', 'Quantidade vendida é zero ou negativa', invalidos_quantidade)

In [ ]:
# Célula 3: Consolidação, Impacto e Priorização

df_relatorio = pd.DataFrame(problemas_encontrados)

# -----------------------------------------------------------------
# Priorização: Ordenar por Criticidade (ALTA > MÉDIA > BAIXA) e Impacto
# -----------------------------------------------------------------
ordem_criticidade = {'ALTA': 3, 'MÉDIA': 2, 'BAIXA': 1}
df_relatorio['Criticidade_Ord'] = df_relatorio['Criticidade'].map(ordem_criticidade)

# Ordena, priorizando problemas de ALTA criticidade e maior impacto
df_relatorio_final = df_relatorio.sort_values(
    by=['Criticidade_Ord', 'Impacto (%)', 'Dataset'], 
    ascending=[False, False, True]
).drop(columns='Criticidade_Ord')

print("--- 🚨 Relatório Consolidado de Problemas de Qualidade 🚨 ---")
print(f"Total de Registros Analisados: Clientes={total_clientes}, Produtos={total_produtos}, Vendas={total_vendas}")

display(df_relatorio_final)

# Célula 4 (Markdown): Conclusão e Próximos Passos

## 🎯 Sumário Executivo e Estratégia de Remediação

O relatório de análise identifica vários pontos de falha na qualidade dos dados que requerem intervenção imediata, especialmente em áreas de **Integridade** e **Unicidade**.

### 1. Problemas de ALTA Criticidade (Foco Imediato no Pipeline de Ingestão)

Estes problemas violam as regras fundamentais de relacionamento (Integridade) ou de identificação (Unicidade), comprometendo a validade de qualquer análise ou relatório que utilize as chaves primárias/estrangeiras.

* **Integridade Referencial (ID Órfão em Vendas):** Vendas sem um `id_cliente` ou `id_produto` válido são inúteis. A primeira linha de defesa deve ser um **teste de integridade referencial** no pipeline de ingestão, rejeitando ou isolando registros que falhem nesta checagem.
* **Unicidade (Clientes Duplicados):** A duplicidade de `id_cliente` (ou a presença de valores nulos em IDs) destrói a confiança na métrica de clientes únicos. O pipeline deve incluir uma etapa de **deduplicação** rigorosa.

### 2. Problemas de MÉDIA Criticidade (Foco em Transformação e Limpeza)

* **Completude (Nome/ID Nulo):** Nomes nulos em produtos e IDs nulos em clientes exigem que a fonte de dados seja corrigida ou que uma regra de imputação (se cabível) seja aplicada.
* **Consistência (Regra de Negócio):** A falha em `valor_total` = `quantidade` × `valor_unitario` exige uma re-cálculo e validação da regra de negócio.

### Próximo Passo: Implementação do Pipeline (Parte 2)

O próximo foco será em utilizar o **Great Expectations (Seção 2.2)** para codificar estas regras de negócio e integridade (unicidade, integridade referencial, validade de preço/quantidade) nas **Expectation Suites**, garantindo que as falhas identificadas neste relatório sejam prevenidas em futuras cargas de dados.